<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Web Scraping

_Author: Joseph Nelson (DC)_

---

## Before Class

### Install Selenium

Students will need to install Selenium using one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`

## Learning Objectives
- Revisit how to locate elements on a webpage
- Aquire unstructure data from the internet using Beautiful soup.
- Discuss limitations associated with simple requests and urllib libraries
- Introduce Selenium as a solution, and implement a scraper using selenium

## Lesson Guide

- [Introduction](#intro)
- [Building a web scraper](#building-scraper)
- [Retrieving data from the HTML page](#retrieving-data)
    - [Retrieving the restaurant names](#retrieving-names)
    - [Challenge: Retrieving the restaurant locations](#retrieving-locations)
    - [Retrieving the restaurant prices](#retrieving-prices)
    - [Retrieving the restaurant number of bookings](#retrieving-bookings)


- [Introducing Selenium](#selenium)
    - [Running JavaScript before scraping](#selenium-js)
    - [Using regex to only get digits](#selenium-regex)
    - [Challenge: Use Pandas to create a DataFrame of bookings](#challenge-pandas)
    - [Auto-typing using Selenium](#selenium-typing)


- [Summary](#summary)

<a id="intro"></a>
## Introduction

In this codealong lesson, we'll build a web scraper using requests and BeautifulSoup. We will also explore how to use a headless browser called Selenium.

We'll begin by scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings

Let's inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

---

<a id="building-scraper"></a>
## Building a web scraper

Now, let's build a web scraper for OpenTable using urllib and Beautiful Soup:

In [1]:
# import our necessary first packages
from bs4 import BeautifulSoup
import requests

In [2]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = requests.get(url)

At this point, what is in html?

In [3]:
# .text returns the request content in Unicode
html.text[:500]

'           <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Restaurant Reservation Availability</title>    <meta  name="robots" content="noindex" > </meta><link  rel="canonical" href="https://www.opentable.com/washington-dc-restaurant-listings" > </link>      <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon.ico" type="image/x-icon"/><link rel="icon" href="/'

We will need to convert this html objct into a soup object so we can parse it using python and BS4

In [4]:
# convert this into a soup object
soup = BeautifulSoup(html.text, 'html.parser')

<a id="retrieving-data"></a>
### Retrieving data from the HTML page

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? (Hint: We need to know where in the **HTML** the restaurant element is housed.) In order to find the HTML that renders the restaurant location, we can use Google Chrome's Inspect tool:

> http://www.opentable.com/washington-dc-restaurant-listings

> 1. Visit the URL above. 

> 2. Right-click on an element you are interested in, then choose Inspect (in Chrome). 

> 3. This will open the Developer Tools and show the HTML used to render the selected page element. 

> Throughout this lesson, we will use this method to find tags associated with elements of the page we want to scrape.

See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [5]:
# print the restaurant names
soup.find_all(name='span', attrs={'class':'rest-row-name-text'})

[<span class="rest-row-name-text">Mission</span>,
 <span class="rest-row-name-text">Gleichner Extension</span>,
 <span class="rest-row-name-text">148 Brekke</span>,
 <span class="rest-row-name-text">Shanahan Turnpike</span>,
 <span class="rest-row-name-text">Possimus Manors</span>,
 <span class="rest-row-name-text">Sed</span>,
 <span class="rest-row-name-text">Voluptatem Falls</span>,
 <span class="rest-row-name-text">Cassandres</span>,
 <span class="rest-row-name-text">Noemie Hamill</span>,
 <span class="rest-row-name-text">Blanditiis</span>,
 <span class="rest-row-name-text">Summit</span>,
 <span class="rest-row-name-text">Rexs</span>,
 <span class="rest-row-name-text">Pike</span>,
 <span class="rest-row-name-text">Ut Circle</span>,
 <span class="rest-row-name-text">Willy Club</span>,
 <span class="rest-row-name-text">Plains</span>,
 <span class="rest-row-name-text">Schultz</span>,
 <span class="rest-row-name-text">Mollitia Brooks</span>,
 <span class="rest-row-name-text">Omnis</span

In [11]:
str(soup.find_all(name='span', attrs={'class':'rest-row-name-text'})[0]

SyntaxError: unexpected EOF while parsing (<ipython-input-11-d965cd66d81c>, line 1)

It is important to always keep in mind the data types that were returned. Note this is a `list`, and we know that immediately by observing the outer square brackets and commas separating each tag.

Next, note the elements of the list are `Tag` objects, not strings. (If they were strings, they would be surrounded by quotes.) The Beautiful Soup authors chose to display a `Tag` object visually as a text representation of the tag and its contents. However, being an object, it has many methods that we can call on it. For example, next we will use the `encode_contents()` method to return the tag's contents encoded as a Python string.

<a id="retrieving-names"></a>
#### Retrieving the restaurant names

Now that we found a list of tags containing the restaurant names, let's think how we can loop through them all one-by-one. In the following cell, we'll print out the name (and **only** the clean name, not the rest of the html) of each restaurant.

In [12]:
# for each element you find, print out the restaurant name
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print(entry.text)

Mission
Gleichner Extension
148 Brekke
Shanahan Turnpike
Possimus Manors
Sed
Voluptatem Falls
Cassandres
Noemie Hamill
Blanditiis
Summit
Rexs
Pike
Ut Circle
Willy Club
Plains
Schultz
Mollitia Brooks
Omnis
Voluptate Trail
58 Raynor
Electas
Et
Leonies
Id Plain
Adams
Esse Neck
Lowe
Eleazar Hyatt
Consectetur
Voluptatum Court
Satterfield
Placeat
Boyle Union
Itaque
Herminias
MacGyver
259 Funk
Perspiciatis Divide
916 Brekke
Ipsam
Beer
Overpass
Landing
Havens
Spring
Hodkiewicz
Rafaela Kub
Fords
Okey Nader
Recusandae Hagenes
Hayden Kozey
Gleason Plains
Beatae Stream
1455 Crona
Hilpert
Expressway
Alexanders
Braun
Est Stanton
Wuckert
Hic
660 Schoen
Iure Spur
Explicabo Walk
Erdman Center
Ipsa Loop
Nam Pfannerstill
Highway
Durgan
Dicta Bernier
Inventore Mews
Laboriosam
Et Gorczany
Valley
Ab
Block
Zulauf
392 Rutherford
Craigs
Kirk Dibbert
Illo Throughway
Pierce Dooley
Lubowitz Creek
Creek
Friedrichs
Schimmel
Sint Overpass
Trenton Forges
1314 Collins
Kuhn
465 Hessel
Als
Soluta Throughway
Brandyn Haye

Great!

<a id="retrieving-locations"></a>
#### Challenge: Retrieving the restaurant locations

Can you repeat that process for finding the location? For example, barmini by Jose Andres is in the location listed as "Penn Quarter" in our search results.

In [ ]:
# first, see if you can identify the location for all elements -- print it out

In [16]:
# now print out EACH location for the restaurants
for entry in soup.find_all(name='span', attrs={'class':'rest-row-meta--location rest-row-meta-text'}):
    print(entry.text)

North Caleighside
Johannaberg
West Santos
New Rahulfort
North Rosa
Satterfieldhaven
North Courtney
Traceymouth
Stantonborough
South Augustus
Rowechester
Armstrongmouth
Kiehnborough
North Jayson
Lake Gracie
Josephbury
North Harrybury
Kayleechester
West Brendon
Lake Matilda
Collierville
South Tracyton
Murphyfort
Dolorestown
D'Amoreshire
Beckerborough
Harberville
West Constance
Port Keshawn
New Cristalshire
South Berneicefurt
Huelschester
Koelpinmouth
Brownview
Hicklefort
East Okeystad
Delphaland
Vickyborough
North Norbertshire
West Melbastad
Brentstad
New Jordyville
Mylenemouth
South Sincere
Ashleemouth
South Lexusville
Bryanaside
South Marcelo
Rutherfordside
New Elsaview
Chasityside
Hellerville
Dachland
West Kelli
Lake Katherinemouth
Edgardobury
New Hilariotown
Batzberg
Port Ruby
New Enrique
North Browntown
Justinaville
Lake Schuylerhaven
Dionside
Port Heatherland
East Floland
Tarynview
Parisianside
Norafort
Jefferyhaven
Koepphaven
East Bernie
Kulasport
Port Megane
Lucienneland
Dickinso

<a id="retrieving-prices"></a>
#### Retrieving the restaurant prices

Ok, we've figured out the restaurant name and location. Now we need to grab the price (number of dollar signs on a scale of one to four) for each restaurant. We'll follow the same process.

In [17]:
# print out all prices
soup.find_all('div', {'class':'rest-row-pricing'})

[<div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    </i> <span class="pricing--not-the-price">  $        </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    </i> <span class="pricing--not-the-price">  $        </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    </i> <span class="pricing--not-the-price">  $        </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    $  </i> <span class="pricing--not-the-price"> </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    $  </i> <span class="pricing--not-the-price"> </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $      </i> <span class="pricing--not-the-price">  $    $      </span></div>,
 <div class="rest-row-pricing"> <i class="pricing--the-price">  $    $    $    </i> <span class="pricing--not-the-price">  $        

In [18]:
# print out EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print(entry.find('i').text)

  $    $    $    
  $    $    $    
  $    $    $    
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $    $    
  $    $    $    
  $    $    $    $  
  $    $    $    
  $    $    $    
  $    $      
  $    $    $    
  $    $    $    $  
  $    $    $    
  $    $    $    
  $    $    $    
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $    $    
  $    $      
  $    $    $    
  $    $    $    
  $    $    $    
  $    $    $    $  
  $    $    $    $  
  $    $    $    
  $    $      
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $    $    
  $    $    $    
  $    $      
  $    $    $    $  
  $    $    $    
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $    $    $  
  $    $    $    
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $    $    
  $    $      
  $    $    $    
  $    $    $

That looks great, but what if I wanted just the number of dollar signs per restaurant? Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [19]:
# print the number of dollars signs per restaurant
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print(entry.find('i').text.count('$'))

3
3
3
4
4
2
3
3
4
3
3
2
3
4
3
3
3
4
4
4
3
2
3
3
3
4
4
3
2
2
4
4
4
4
3
3
2
4
3
4
4
4
4
4
4
3
2
2
4
2
3
2
3
3
3
3
2
2
4
3
4
2
4
4
3
4
4
2
3
2
3
4
3
4
2
3
3
2
2
2
4
3
3
2
3
4
2
4
4
4
3
4
3
3
2
3
2
4
3
4


Phew, nice work. 

<a id="retrieving-bookings"></a>
#### Retrieving the restaurant number of bookings

One more, right? We only need to find the number times a restaurant was booked. In the next cell, print out all objects that contain the number of times the restaurant was booked.

In [20]:
# print out all objects that contain the number of times the restaurant was booked
soup.find_all('div', {'class':'booking'})

[]

That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

In [22]:
# let's first try printing out all 'div' objects
#  NOTE: This is a too many objects to store in this notebook!
#        So, uncomment the code below to run it.

# for entry in soup.find_all('div'):
#     print(entry)

I still don't see it. Let's search our entire soup object:

In [23]:
# print out soup, do command+f for "booked ". 
#   Uncomment the below to run.

soup

 <!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="IE=9; IE=8; IE=7; IE=EDGE" http-equiv="X-UA-Compatible"/> <title>Restaurant Reservation Availability</title> <meta content="noindex" name="robots"/> <link href="https://www.opentable.com/washington-dc-restaurant-listings" rel="canonical"/> <link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-16.png" rel="icon" sizes="16x16"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-32.png" rel="icon" sizes="32x32"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-48.png" rel="icon" sizes="48x48"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-64.png" rel="icon" sizes="64x64"/><link href="//components.otstatic.com/components/favicon/1.0.5/favicon/favicon-128.png" rel="icon" sizes="

What do you notice? Why is this happening?

<a id="selenium"></a>
## Introducing Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

In [25]:
# import
# conda install seleniums
from selenium import webdriver

In [ ]:
# STOP
# what is going to happen when I run the next cell?

In [28]:
# create a driver that will execute in Google Chrome
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

Pretty crazy, right? Let's close that driver.

In [29]:
# close it
driver.close()

In [30]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. In the next cell, prove you can programmatically visit the page.

In [31]:
# visit our OpenTable page
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

In [32]:
driver.title

'Washington, D.C. Area Restaurants List | OpenTable'

In [33]:
driver.close()

<a id="selenium-js"></a>
### Running JavaScript before scraping

Now, to resolve our JavaScript problem, there's a few things we can do. What I'll do in this case is request that the page load, wait one second, and then I'm going to grab the source html from the page. Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

In [34]:
# import sleep
from time import sleep

In [35]:
# visit our relevant page
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# wait one second
sleep(1)

#grab the page source
html = driver.page_source

**Pop Quiz:** What do we need to do with this HTML?

In [36]:
# BeautifulSoup it!
html = BeautifulSoup(html, "lxml")

Now, let's return to our earlier problem: How do we locate bookings on the page?

In [37]:
# print out the number bookings for all restaurants
html.find_all('div', {'class':'booking'})

[<div class="booking"><span class="tadpole"></span>Booked 2 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 646 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 352 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 185 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 124 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 96 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 39 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 81 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 204 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 169 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 88 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 38 times today</div>,
 <div class="booking"><span class="

In [ ]:
# now print out each booking for the listings using a loop
for entry in html.find_all('div', {'class':'booking'}):
    print(entry)

Let's grab just the text of each of these entries.

In [38]:
# do the same as above, but grabbing only the text content
for entry in html.find_all('div', {'class':'booking'}):
    print(entry.text)

Booked 2 times today
Booked 646 times today
Booked 352 times today
Booked 185 times today
Booked 124 times today
Booked 96 times today
Booked 39 times today
Booked 81 times today
Booked 204 times today
Booked 169 times today
Booked 88 times today
Booked 38 times today
Booked 65 times today
Booked 18 times today
Booked 59 times today
Booked 28 times today
Booked 39 times today
Booked 25 times today
Booked 70 times today
Booked 55 times today
Booked 56 times today
Booked 77 times today
Booked 24 times today
Booked 52 times today
Booked 95 times today
Booked 68 times today
Booked 39 times today
Booked 23 times today
Booked 33 times today
Booked 18 times today
Booked 64 times today
Booked 62 times today
Booked 125 times today
Booked 36 times today
Booked 157 times today
Booked 14 times today
Booked 107 times today
Booked 149 times today
Booked 75 times today
Booked 15 times today
Booked 11 times today
Booked 60 times today
Booked 49 times today
Booked 78 times today
Booked 8 times today
Bo

In [39]:
driver.close()

We've succeeded!

<a id="selenium-regex"></a>
### Using regex to only get digits

But we can clean this up a little bit. We're going to use regular expressions (regex) to grab only the digits that are available in each of the text.

The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [40]:
# import regex
import re

Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [41]:
# for each entry, grab the text
for booking in html.find_all('div', {'class':'booking'}):
    # match all digits
    match = re.search('\d+', booking.text)
    
    if match:
        # print if found
        print(match.group())
    else:
        # otherwise pass
        pass

2
646
352
185
124
96
39
81
204
169
88
38
65
18
59
28
39
25
70
55
56
77
24
52
95
68
39
23
33
18
64
62
125
36
157
14
107
149
75
15
11
60
49
78
8
47
15
46
40
50
11
21
64
14
24
33
18
18
57
39
13
49
6
16
17
37
57
72
12
28
58
33
52
39
2
38
18
10
43
26
12
19
39
48
37
68
5
20
29
1
5
70
49
11
7
24
38
22
36


Before we demonstrate all the other amazing things about headless browsers, let's finish up collecting the data we want from this current example. Do you suppose the html parsing we wrote above will still work on the page source we've grabbed from our headless browser?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [ ]:
# print out all entries
#   NOTE: Has many entries. Uncomment the below code to run it!

# soup.find_all('div', {'class':'result content-section-list-row cf with-times'})

Look over the page. Does every single entry have each element we're seeking?
> I did this previously. I know for a fact that not every element has a number of recent bookings. That's probably exactly why OpenTable houses this in JavaScript: they want to continously update the number of bookings with the most relevant number of values.

In [42]:
# What happens when a booking is not available?
# Print out each booking entry, using the identification code we wrote above

for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print(entry.find('div', {'class':'booking'}))

<div class="booking"><span class="tadpole"></span>Booked 2 times today</div>
None
<div class="booking"><span class="tadpole"></span>Booked 646 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 352 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 185 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 124 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 96 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 39 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 81 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 204 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 169 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 88 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 38 times today</div>
<div class="booking"><span class="tadpole"></span>Book

In [43]:
# We want to only retrieve the text of the bookings.
# But what would happen if we just naively print the text of each node?
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print(entry.find('div', {'class':'booking'}).text)   # try adding .text

Booked 2 times today


AttributeError: 'NoneType' object has no attribute 'text'

What do you notice takes the place when booking is not found?

We could use exception handling (`try`/`except` blocks) to resolve this. However, exceptions should only be used to handle rare or unexpected errors -- never for normal program flow.

In this case, we expect that some entries will be zero. So, we can just use an `if` statement that tests whether there are any `divs` present; if not, display `'ZERO'`. Here's a demo:

In [44]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    booking_tag = entry.find('div', {'class':'booking'})
    
    if booking_tag:
        print(booking_tag.text)
    else:
        print('ZERO')

Booked 2 times today
ZERO
Booked 646 times today
Booked 352 times today
Booked 185 times today
Booked 124 times today
Booked 96 times today
Booked 39 times today
Booked 81 times today
Booked 204 times today
Booked 169 times today
Booked 88 times today
Booked 38 times today
Booked 65 times today
Booked 18 times today
Booked 59 times today
Booked 28 times today
Booked 39 times today
Booked 25 times today
Booked 70 times today
Booked 55 times today
Booked 56 times today
Booked 77 times today
Booked 24 times today
Booked 52 times today
Booked 95 times today
Booked 68 times today
Booked 39 times today
Booked 23 times today
Booked 33 times today
Booked 18 times today
Booked 64 times today
Booked 62 times today
Booked 125 times today
Booked 36 times today
Booked 157 times today
Booked 14 times today
Booked 107 times today
Booked 149 times today
Booked 75 times today
Booked 15 times today
Booked 11 times today
Booked 60 times today
Booked 49 times today
Booked 78 times today
Booked 8 times tod

After previously completing this, we observed that all other elements WILL be returned. This means we do not have to always handle these cases.

<a id="challenge-pandas"></a>
### Challenge: Use Pandas to create a DataFrame of bookings

However, the onus is on you to now put all the pieces together.

Loop through each entry. For each entry, grab the relevant information we want (name, location, price, bookings). Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [49]:
import pandas as pd

In [50]:
# I'm going to create my empty df first
dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

In [55]:
# Put code here that populates the DataFrame using Selenium and BeautifulSoup!
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    booking_tag = entry.find('div', {'class':'booking'})
    bookings= 'NA'
    # get bookings
    if booking_tag:
        match = re.search('\d+', booking_tag.text)
        if match:
            bookings = match.group()
    # name
    name =  entry.find('span', {'class':'rest-row-name-text'}).text
    location =  entry.find('span', {'class':'rest-row-meta--location rest-row-meta-text'}).text
    price =  entry.find('div', {'class':'rest-row-pricing'}).find('i').text.count('$')
    
    result = {'price': price, 'location': location, 'name': name, 'bookings': bookings}
    dc_eats.append(result,  ignore_index=True) 

In [56]:
# check out our work
dc_eats.head()

,name,location,price,bookings
0,Ruffino's - Arlington,Arlington,2,2
1,Joe's Place Pizza and Pasta,Arlington,2,NA
2,Founding Farmers - DC,Foggy Bottom,2,646
3,Farmers Fishers Bakers,Georgetown,2,352
4,Filomena Ristorante,Georgetown,3,185


Awesome! We succeeded.

<a id="selenium-typing"></a>
### Auto-typing using Selenium

Now, let's explore some of the other functionality of a webdriver. We've barely scratched the surface.

In [58]:
# we can send keys as well

from selenium.webdriver.common.keys import Keys

In [59]:
# open Chrome
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

# visit Python
driver.get("http://www.python.org")

# verify we're in the right place
assert "Python" in driver.title

Let's try automatedly typing `pycon` in the search box and hitting the return key:

In [60]:
# find the search position
elem = driver.find_element_by_name("q")

# clear it
elem.clear()

# type in pycon
elem.send_keys("pycon")

# send those keys
elem.send_keys(Keys.RETURN)

In [61]:
# close
driver.close()

In [63]:
# all at once:
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver')
driver.get("http://www.python.org")
assert "Python" in driver.title

elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(" asdfasdfasd pycon")
elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
# driver.close()

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html

### Summary

In this lesson, we used the Beautiful Soup library to locate elements on a website then scrape their text. We also used the Selenium headless browser to run JavaScript first before retrieving the page contents.